# Trend Analysis - Energy Data

**Purpose**: Analyze time series trends, forecasting patterns, and seasonal analysis

**Date**: January 12, 2026

## Objectives
1. Verify data quality before analysis
2. Analyze overall energy trends (total, renewable, fossil)
3. Calculate country-level trend metrics
4. Identify forecasting patterns
5. Visualize trend distributions and relationships

## ⚠️ IMPORTANT: Data Quality Check Required
Before performing trend analysis, we must verify:
- Data completeness (no missing years)
- Fossil energy data availability (2016-2017 had issues)
- Renewable percentage validity (not all 100%)
- Outlier detection and handling

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Database connection
DB_CONFIG = {
    'host': '172.18.0.1',
    'port': 5432,
    'database': 'lianel_energy',
    'user': 'airflow',
    'password': 'P9xK2mN7vQ4wR8tY3sL6hJ5nB1cV0zX'
}

connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
engine = create_engine(connection_string)

print("✅ Database connection established")

## 1. Data Quality Verification

**CRITICAL**: Before trend analysis, verify data quality to ensure reliable results.

In [ ]:
# Load ML forecasting dataset
query = """
SELECT 
    cntr_code,
    year,
    total_energy_gwh,
    renewable_energy_gwh,
    fossil_energy_gwh,
    pct_renewable,
    pct_fossil,
    yoy_change_total_energy_pct,
    yoy_change_renewable_pct,
    energy_density_gwh_per_km2,
    area_km2
FROM ml_dataset_forecasting_v1
ORDER BY cntr_code, year
"""

df = pd.read_sql(query, engine)
print(f"✅ Loaded {len(df)} records")
print(f"Countries: {df['cntr_code'].nunique()}")
print(f"Years: {df['year'].min()} - {df['year'].max()}")
print(f"\nData shape: {df.shape}")
df.head()

In [ ]:
# DATA QUALITY CHECKS

quality_issues = []
quality_warnings = []

print("🔍 Data Quality Verification:")
print("=" * 80)

# Check 1: Check for missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    quality_issues.append(f"Missing values detected: {missing[missing > 0].to_dict()}")
    print(f"❌ Missing values: {missing[missing > 0].to_dict()}")
else:
    print("✅ No missing values")

# Check 2: Verify fossil energy data completeness by year
completeness_by_year = df.groupby('year').agg({
    'fossil_energy_gwh': lambda x: (x > 0).sum(),
    'cntr_code': 'count'
}).rename(columns={'fossil_energy_gwh': 'has_fossil_data', 'cntr_code': 'total_records'})
completeness_by_year['fossil_completeness_pct'] = (completeness_by_year['has_fossil_data'] / completeness_by_year['total_records']) * 100

print("\n📊 Fossil Energy Data Completeness by Year:")
print(completeness_by_year.to_string())

# Flag incomplete years (2016-2017 had known issues)
incomplete_years = completeness_by_year[completeness_by_year['fossil_completeness_pct'] < 90].index.tolist()
if incomplete_years:
    quality_warnings.append(f"Incomplete fossil data in years: {incomplete_years}")
    print(f"\n⚠️  WARNING: Incomplete fossil data in years: {incomplete_years}")
    print("   These years should be excluded or flagged in analysis")
else:
    print("\n✅ All years have >90% fossil data completeness")

# Check 3: Verify renewable percentages are realistic (not all 100%)
suspicious_renewable = df[df['pct_renewable'] >= 99]
suspicious_count = len(suspicious_renewable)
if suspicious_count > len(df) * 0.1:  # More than 10% showing 100% renewable
    quality_issues.append(f"{suspicious_count} records show 100% renewable (likely data quality issue)")
    print(f"\n❌ {suspicious_count} records ({suspicious_count/len(df)*100:.1f}%) show ≥99% renewable")
    print("   This suggests missing fossil data")
    print("\n   Sample of suspicious records:")
    print(suspicious_renewable[['cntr_code', 'year', 'pct_renewable', 'fossil_energy_gwh', 'total_energy_gwh']].head(10).to_string(index=False))
else:
    print(f"\n✅ Renewable percentages look realistic ({suspicious_count} records with ≥99% renewable)")

# Check 4: Check for extreme YoY changes (indicates data quality issues)
extreme_yoy = df[df['yoy_change_total_energy_pct'].abs() > 200]
if len(extreme_yoy) > 0:
    quality_warnings.append(f"{len(extreme_yoy)} records with extreme YoY changes (>200%)")
    print(f"\n⚠️  WARNING: {len(extreme_yoy)} records with extreme YoY changes (>200%)")
    print("   These may indicate data quality issues or missing baseline data")
    print("\n   Sample of extreme changes:")
    print(extreme_yoy[['cntr_code', 'year', 'yoy_change_total_energy_pct', 'total_energy_gwh']].head(10).to_string(index=False))

# Check 5: Verify data coverage (countries per year)
coverage_by_year = df.groupby('year')['cntr_code'].nunique()
print(f"\n📊 Country Coverage by Year:")
print(coverage_by_year.to_string())
if coverage_by_year.std() > coverage_by_year.mean() * 0.1:
    quality_warnings.append("Inconsistent country coverage across years")
    print("\n⚠️  WARNING: Inconsistent country coverage across years")

# Summary
print("\n" + "=" * 80)
print("📋 Data Quality Summary:")
print(f"  Total records: {len(df)}")
print(f"  Countries: {df['cntr_code'].nunique()}")
print(f"  Years: {df['year'].min()} - {df['year'].max()}")
print(f"  Quality issues: {len(quality_issues)}")
print(f"  Quality warnings: {len(quality_warnings)}")

if quality_issues:
    print("\n🔴 CRITICAL ISSUES FOUND:")
    for issue in quality_issues:
        print(f"  - {issue}")
    print("\n⚠️  RECOMMENDATION: Fix data quality issues before proceeding with trend analysis")
elif quality_warnings:
    print("\n⚠️  WARNINGS:")
    for warning in quality_warnings:
        print(f"  - {warning}")
    print("\n💡 RECOMMENDATION: Review warnings and consider filtering incomplete data")
else:
    print("\n✅ Data quality checks passed!")

## 2. Data Filtering & Preparation

Filter out incomplete years and prepare clean dataset for trend analysis.

In [ ]:
# Filter data based on quality checks
# Exclude years with incomplete fossil data (2016-2017 had known issues)
# Also exclude records with suspicious renewable percentages or extreme YoY changes

print("🔧 Data Filtering:")
print("=" * 80)

# Option 1: Filter to years with complete data (recommended)
# Based on completeness check, filter to years with >90% fossil data completeness
complete_years = completeness_by_year[completeness_by_year['fossil_completeness_pct'] >= 90].index.tolist()
df_filtered = df[df['year'].isin(complete_years)].copy()

print(f"Original records: {len(df)}")
print(f"Filtered records (complete years only): {len(df_filtered)}")
print(f"Excluded years: {sorted(set(df['year']) - set(complete_years))}")
print(f"Years included: {sorted(complete_years)}")

# Option 2: Additional filtering for extreme outliers (optional)
# Remove records with extreme YoY changes that likely indicate data errors
df_filtered = df_filtered[df_filtered['yoy_change_total_energy_pct'].abs() <= 200].copy()
print(f"\nAfter removing extreme YoY outliers (>200%): {len(df_filtered)} records")

# Verify filtered data
print(f"\n✅ Filtered dataset:")
print(f"  Countries: {df_filtered['cntr_code'].nunique()}")
print(f"  Years: {df_filtered['year'].min()} - {df_filtered['year'].max()}")
print(f"  Records per country: {df_filtered.groupby('cntr_code').size().describe().to_string()}")

# Use filtered data for analysis
df = df_filtered.copy()
print("\n✅ Data prepared for trend analysis")

## 3. Trend Calculation Functions

In [ ]:
def calculate_trend(x, y):
    """
    Calculate linear trend (slope) and correlation coefficient (R) for time series data.
    
    Parameters:
    - x: time values (e.g., years)
    - y: values to analyze (e.g., energy consumption)
    
    Returns:
    - (slope, correlation_coefficient) or (None, None) if insufficient data
    """
    # Remove NaN values
    mask = ~(np.isnan(x) | np.isnan(y))
    x_clean = x[mask]
    y_clean = y[mask]
    
    if len(x_clean) < 2:
        return None, None
    
    # Calculate linear regression
    slope, intercept, r_value, p_value, std_err = stats.linregress(x_clean, y_clean)
    
    return slope, r_value

print("✅ Trend calculation function defined")

## 4. Overall Trends

In [ ]:
# Calculate overall trends (aggregated across all countries)
overall = df.groupby('year').agg({
    'total_energy_gwh': 'sum',
    'renewable_energy_gwh': 'sum',
    'fossil_energy_gwh': 'sum',
    'pct_renewable': 'mean'
}).reset_index()

# Calculate trends
total_trend = calculate_trend(overall['year'], overall['total_energy_gwh'])
renewable_trend = calculate_trend(overall['year'], overall['renewable_energy_gwh'])
fossil_trend = calculate_trend(overall['year'], overall['fossil_energy_gwh'])
renewable_pct_trend = calculate_trend(overall['year'], overall['pct_renewable'])

print("📊 Overall Energy Trends (All Countries Combined):")
print("=" * 80)
print(f"Total Energy:      Slope = {total_trend[0]:.2f} GWh/year, R² = {total_trend[1]**2:.3f}")
print(f"Renewable Energy: Slope = {renewable_trend[0]:.2f} GWh/year, R² = {renewable_trend[1]**2:.3f}")
print(f"Fossil Energy:    Slope = {fossil_trend[0]:.2f} GWh/year, R² = {fossil_trend[1]**2:.3f}")
print(f"Renewable %:      Slope = {renewable_pct_trend[0]:.3f} %/year, R² = {renewable_pct_trend[1]**2:.3f}")

# Visualize overall trends
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Total energy over time
ax1 = axes[0, 0]
ax1.plot(overall['year'], overall['total_energy_gwh'], marker='o', linewidth=2, markersize=8, label='Total Energy')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Energy (GWh)')
ax1.set_title(f'Total Energy Consumption Trend\n(Slope: {total_trend[0]:.0f} GWh/year, R²: {total_trend[1]**2:.3f})')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Renewable vs Fossil
ax2 = axes[0, 1]
ax2.plot(overall['year'], overall['renewable_energy_gwh'], marker='o', linewidth=2, markersize=8, label='Renewable', color='green')
ax2.plot(overall['year'], overall['fossil_energy_gwh'], marker='s', linewidth=2, markersize=8, label='Fossil', color='red')
ax2.set_xlabel('Year')
ax2.set_ylabel('Energy (GWh)')
ax2.set_title('Renewable vs Fossil Energy')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Plot 3: Renewable percentage
ax3 = axes[1, 0]
ax3.plot(overall['year'], overall['pct_renewable'], marker='o', linewidth=2, markersize=8, color='green')
ax3.set_xlabel('Year')
ax3.set_ylabel('Renewable Percentage (%)')
ax3.set_title(f'Renewable Energy Percentage Trend\n(Slope: {renewable_pct_trend[0]:.3f} %/year, R²: {renewable_pct_trend[1]**2:.3f})')
ax3.grid(True, alpha=0.3)
ax3.axhline(y=50, color='orange', linestyle='--', linewidth=1, alpha=0.5, label='50% threshold')
ax3.legend()

# Plot 4: Energy mix pie chart (latest year)
ax4 = axes[1, 1]
latest_year = overall['year'].max()
latest_data = overall[overall['year'] == latest_year].iloc[0]
renewable_pct = (latest_data['renewable_energy_gwh'] / latest_data['total_energy_gwh']) * 100
fossil_pct = (latest_data['fossil_energy_gwh'] / latest_data['total_energy_gwh']) * 100
other_pct = 100 - renewable_pct - fossil_pct

ax4.pie([renewable_pct, fossil_pct, other_pct], 
        labels=[f'Renewable ({renewable_pct:.1f}%)', f'Fossil ({fossil_pct:.1f}%)', f'Other ({other_pct:.1f}%)'],
        colors=['green', 'red', 'gray'],
        autopct='%1.1f%%',
        startangle=90)
ax4.set_title(f'Energy Mix - {int(latest_year)}')

plt.tight_layout()
plt.show()

## 5. Country-Level Trend Analysis

In [ ]:
# Calculate trends for each country
country_trends = []
for country in df['cntr_code'].unique():
    country_data = df[df['cntr_code'] == country].sort_values('year')
    if len(country_data) >= 3:  # Need at least 3 points for trend
        total_trend = calculate_trend(country_data['year'], country_data['total_energy_gwh'])
        renewable_trend = calculate_trend(country_data['year'], country_data['renewable_energy_gwh'])
        renewable_pct_trend = calculate_trend(country_data['year'], country_data['pct_renewable'])
        
        country_trends.append({
            'cntr_code': country,
            'total_slope': total_trend[0] if total_trend[0] else 0,
            'total_r2': total_trend[1]**2 if total_trend[1] else 0,
            'renewable_slope': renewable_trend[0] if renewable_trend[0] else 0,
            'renewable_r2': renewable_trend[1]**2 if renewable_trend[1] else 0,
            'renewable_pct_slope': renewable_pct_trend[0] if renewable_pct_trend[0] else 0,
            'renewable_pct_r2': renewable_pct_trend[1]**2 if renewable_pct_trend[1] else 0,
            'avg_total_energy': country_data['total_energy_gwh'].mean(),
            'avg_renewable_pct': country_data['pct_renewable'].mean()
        })

trends_df = pd.DataFrame(country_trends)

print("📊 Country-Level Trend Analysis:")
print("\nTop 10 Countries by Renewable Percentage Growth:")
top_renewable_growth = trends_df.nlargest(10, 'renewable_pct_slope')
print(top_renewable_growth[['cntr_code', 'renewable_pct_slope', 'renewable_pct_r2', 'avg_renewable_pct']].to_string(index=False))

print("\nTop 10 Countries by Total Energy Growth:")
top_energy_growth = trends_df.nlargest(10, 'total_slope')
print(top_energy_growth[['cntr_code', 'total_slope', 'total_r2', 'avg_total_energy']].to_string(index=False))

# Visualize country trends
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Renewable percentage trends by country (top 10)
ax1 = axes[0, 0]
top_countries = trends_df.nlargest(10, 'renewable_pct_slope')['cntr_code']
for country in top_countries:
    country_data = df[df['cntr_code'] == country].sort_values('year')
    ax1.plot(country_data['year'], country_data['pct_renewable'], 
             marker='o', label=country, linewidth=2, markersize=6)
ax1.set_xlabel('Year')
ax1.set_ylabel('Renewable Percentage (%)')
ax1.set_title('Top 10 Countries by Renewable Growth')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax1.grid(True, alpha=0.3)

# Plot 2: Total energy trends (top 10 by growth)
ax2 = axes[0, 1]
top_energy = trends_df.nlargest(10, 'total_slope')['cntr_code']
for country in top_energy:
    country_data = df[df['cntr_code'] == country].sort_values('year')
    ax2.plot(country_data['year'], country_data['total_energy_gwh'], 
             marker='o', label=country, linewidth=2, markersize=6)
ax2.set_xlabel('Year')
ax2.set_ylabel('Total Energy (GWh)')
ax2.set_title('Top 10 Countries by Energy Growth')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
ax2.grid(True, alpha=0.3)

# Plot 3: Renewable percentage slope distribution
ax3 = axes[1, 0]
ax3.hist(trends_df['renewable_pct_slope'], bins=20, alpha=0.7, edgecolor='black', color='green')
ax3.axvline(x=0, color='red', linestyle='--', linewidth=2, label='No change')
ax3.set_xlabel('Renewable % Slope (%/year)')
ax3.set_ylabel('Number of Countries')
ax3.set_title('Distribution of Renewable Percentage Growth Rates')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Scatter: Renewable % vs Growth Rate
ax4 = axes[1, 1]
ax4.scatter(trends_df['avg_renewable_pct'], trends_df['renewable_pct_slope'], 
           alpha=0.6, s=100, c=trends_df['renewable_pct_r2'], cmap='viridis')
ax4.set_xlabel('Average Renewable Percentage (%)')
ax4.set_ylabel('Renewable Growth Rate (%/year)')
ax4.set_title('Renewable % vs Growth Rate (color = R²)')
ax4.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax4.grid(True, alpha=0.3)
cbar = plt.colorbar(ax4.collections[0], ax=ax4)
cbar.set_label('R² (Trend Strength)')

plt.tight_layout()
plt.show()